<a href="https://colab.research.google.com/github/SILVIAIRENE/Data-Scientist-Machine-Learning-Engineer-Introductory-Course/blob/master/Estudios_de_cr%C3%A9dito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Cargar los datos
train_df = pd.read_csv('application_train.csv')

# Preprocesamiento: Seleccionar un subconjunto de características y la variable objetivo
# Para el modelo base, usaremos solo las características numéricas y una pequeña parte de los datos
numerical_features = train_df.select_dtypes(include=np.number).columns.tolist()
if 'TARGET' in numerical_features:
    numerical_features.remove('TARGET')

# Separar características (X) y la variable objetivo (y)
X = train_df[numerical_features]
y = train_df['TARGET']

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Manejo de valores perdidos: Imputación con la media
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

# Entrenar el modelo de Regresión Logística
model = LogisticRegression(solver='liblinear', random_state=42)
model.fit(X_train_scaled, y_train)

# Hacer predicciones en el conjunto de validación
y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]

# Evaluar con la métrica AUC-ROC
auc_roc = roc_auc_score(y_val, y_pred_proba)
print(f"AUC-ROC del modelo base en el conjunto de validación: {auc_roc:.4f}")
# Cargar los datos de prueba
test_df = pd.read_csv('application_test.csv')
test_ids = test_df['SK_ID_CURR']

# Preparar los datos de prueba
X_test = test_df[numerical_features]
X_test_imputed = imputer.transform(X_test)
X_test_scaled = scaler.transform(X_test_imputed)

# Realizar predicciones
predictions = model.predict_proba(X_test_scaled)[:, 1]

# Crear el archivo de envío
submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': predictions})
submission.to_csv('submission_base_model.csv', index=False)

print("Archivo de envío 'submission_base_model.csv' creado exitosamente.")
from sklearn.preprocessing import OneHotEncoder

# Identificar características categóricas
categorical_features = train_df.select_dtypes(include='object').columns.tolist()

# Crear una instancia de OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Ajustar y transformar en los datos de entrenamiento
encoded_features_train = encoder.fit_transform(train_df[categorical_features])
encoded_features_train_df = pd.DataFrame(encoded_features_train, columns=encoder.get_feature_names_out(categorical_features))

# Unir con los datos numéricos
X_train_encoded = pd.concat([X_train.reset_index(drop=True), encoded_features_train_df], axis=1)



AUC-ROC del modelo base en el conjunto de validación: 0.7384
Archivo de envío 'submission_base_model.csv' creado exitosamente.
